### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

**Question 1:** To capture some other aspects of dates, create columns in your dataset that capture the following aspects of each timestamp:

  - What quarter it's in
  - What month it's in
  - What year it's in
  - The number of days passed in the `visit_date` column

If you want to try adding different pandas date parts, you can find them here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [ ]:
# your answer here
import pandas as pd
import numpy as np

# you might have to change the file path
df = pd.read_csv('../../data/restaurant_data/master.csv', parse_dates=['visit_date', 'calendar_date'])
# sorting the values by the dates before you do this is not a bad idea
df.sort_values(by=['id', 'visit_date'], inplace=True)

In [ ]:
df['quarter'] = df['visit_date'].dt.quarter
df['month']   = df['visit_date'].dt.month
df['year']    = df['visit_date'].dt.year
df['time']    = (df['visit_date'] - df['visit_date'].min()).dt.days

**Question 2:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

However, your data won't really "know" about those values unless they can be observed alongside the current observation.

To that end, make three columns that capture the value of the following:

 - What the previous recorded attendance for the previous observation
 - The attendance from two observations ago
 - The attendance from 7 observations ago (ie, week over week)

In [ ]:
# your answer here
df['yesterday']    = df.groupby('id')['visitors'].shift()
df['two_days_ago'] = df.groupby('id')['visitors'].shift(2)
df['one_week_ago'] = df.groupby('id')['visitors'].shift(7)

**Bonus Answer:** 

In [ ]:
# create the date offsets
one_day_ago  = pd.DateOffset(days=1)
two_days_ago = pd.DateOffset(days=2)
one_week_ago = pd.DateOffset(weeks=1)

In [ ]:
# and the groupings
one_day_shift  = df.set_index('visit_date').groupby('id')[['visitors']].shift(freq=one_day_ago).rename({'visitors': 'one_day_ago'}, axis=1)
two_days_shift = df.set_index('visit_date').groupby('id')[['visitors']].shift(freq=two_days_ago).rename({'visitors': 'two_days_ago_'}, axis=1)
one_week_shift = df.set_index('visit_date').groupby('id')[['visitors']].shift(freq=one_week_ago).rename({'visitors': 'one_week_ago_'}, axis=1)

In [ ]:
# merge them back in 
df = df.merge(one_day_shift, left_on=['id', 'visit_date'], right_index=True, how='left')
df = df.merge(two_days_shift, left_on=['id', 'visit_date'], right_index=True, how='left')
df = df.merge(one_week_shift, left_on=['id', 'visit_date'], right_index=True, how='left')

In [ ]:
# last three columns are the new ones we created -- might want to rename for clarity
df

**Question 3:** Window Statistics

Lots of times, we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [ ]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.head()

In [ ]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 7, 25, and 60 day moving averages for visits for each restaurant inside the dataset.

And be mindful of performing these on the appropriate levels of your dataset.

In [ ]:
# your answer here
df['rolling_mean_7']  = df.groupby('id')['visitors'].rolling(7).mean().shift().values
df['rolling_mean_25'] = df.groupby('id')['visitors'].rolling(25).mean().shift().values
df['rolling_mean_60'] = df.groupby('id')['visitors'].rolling(60).mean().shift().values

In [ ]:
# our final dataset
df

One additional note:  for a calculation such as this is best if you shift the values up by one -- why might this be the case?